# Pricing Test

## Experiment Description
Company XYZ sells a software for \$39. Since revenue has been flat for some time, the VP of Product has decided to run a test increasing the price. She hopes that this would increase revenue. In the experiment, 66% of the users have seen the old price (\$39), while a random sample of 33% users a higher price (\$59). The test has been running for some time and the VP of Product is interested in understanding how it went and whether it would make sense to increase the price for all the users.

**Null Hypothesis** : The null hypothesis is that this increasing price might lost customers and might not be effective in increasing the revenue.

***Alternative Hypothesis*** : The alternative hypothesis is that this might reduce the number of customers who purchase the software, but increasing the overall revenue.

## Experimental Design
Metric Choice
Invariant Metrics : source.

Evaluation Metrics : conversion, gross revenue, net revenue.

## Measuring Standard Deviation


In [86]:
# load modules
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import scipy.stats

In [87]:
# load data
df_results = pd.read_csv('https://github.com/jyan83/Data_Challenges/blob/master/Costly%20Conversion/test_results.csv?raw=true')
df_user = pd.read_csv('https://github.com/jyan83/Data_Challenges/blob/master/Costly%20Conversion/user_table.csv?raw=true')

# join this two tables
df = df_results.set_index('user_id').join(df_user.set_index('user_id'), on = 'user_id')
df.head()

,timestamp,source,device,operative_system,test,price,converted,city,country,lat,long
user_id,,,,,,,,,,,
604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0,Buffalo,USA,42.89,-78.86
624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0,Lakeville,USA,44.68,-93.24
317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0,Parma,USA,41.38,-81.73
685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59,0,Fayetteville,USA,35.07,-78.90
820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39,0,Fishers,USA,39.95,-86.02


In [88]:
# check the control and experiment groups
print(df['test'].value_counts(normalize=True))
df.groupby(['test']).count()

0    0.639921
1    0.360079
Name: test, dtype: float64


,timestamp,source,device,operative_system,price,converted,city,country,lat,long
test,,,,,,,,,,
0,202727,202727,202727,202727,202727,202727,176428,176428,176428,176428
1,114073,114073,114073,114073,114073,114073,99188,99188,99188,99188


The actual percentage of the experiment group is 64\%, slightly smaller than the experiment design value 66\%. There might be some errors in the datasets.

In [89]:
# check the timestamp of the two groups starting from the same day
print('experiment timestamp: ', df[df['test']==1].timestamp.min())
print('control timestamp: ', df[df['test']==0].timestamp.min())

experiment timestamp:  2015-03-02 00:04:12
control timestamp:  2015-03-02 00:12:40


In [90]:
# Add a column of the evaluation metric: net revenue
df['net_revenue'] = df['converted']*df['price']

## Measuring Standard Deviation


In [91]:
# Found out the probability of the evalutaion metric
df[df['test']==1].converted.value_counts(normalize=True)
#round(np.sqrt((.36*(1-.36))/(114073)),4)
df[df['test']==1].net_revenue.value_counts(normalize=True)


0     0.984457
59    0.015534
39    0.000009
Name: net_revenue, dtype: float64

In [92]:
# set the creiteris for two groups of dataframe
df_exp = df[(df['test'] == 1) & (df['price'] == 59)]
df_con = df[(df['test'] == 0) & (df['price'] == 39)]

# get the total number of people in the control and experiment group
N_exp = len(df_exp)
N_con = len(df_con)

df_exp.describe()

,test,price,converted,lat,long,net_revenue
count,113918.0,113918.0,113918.000000,99053.000000,99053.000000,113918.000000
mean,1.0,59.0,0.015555,37.138323,-93.974115,0.917748
std,0.0,0.0,0.123747,5.233333,18.076179,7.301050
min,1.0,59.0,0.000000,19.700000,-157.800000,0.000000
25%,1.0,59.0,0.000000,33.660000,-112.200000,0.000000
50%,1.0,59.0,0.000000,37.770000,-89.060000,0.000000
75%,1.0,59.0,0.000000,40.720000,-79.020000,0.000000
max,1.0,59.0,1.000000,61.180000,30.310000,59.000000


In [93]:
df_con.describe()

,test,price,converted,lat,long,net_revenue
count,202517.0,202517.0,202517.000000,176241.000000,176241.000000,202517.000000
mean,0.0,39.0,0.019900,37.097212,-93.985221,0.776083
std,0.0,0.0,0.139656,5.196415,18.092998,5.446566
min,0.0,39.0,0.000000,19.700000,-157.800000,0.000000
25%,0.0,39.0,0.000000,33.660000,-112.240000,0.000000
50%,0.0,39.0,0.000000,37.720000,-88.550000,0.000000
75%,0.0,39.0,0.000000,40.700000,-78.910000,0.000000
max,0.0,39.0,1.000000,61.180000,30.310000,39.000000


## Sanity Check
This check is primarily for the invariant metrics. For invariant metrics we expect equal diversion into the experiment and control group. We will test this at the 95% confidence interval.

In [94]:
# one hot encoding the control table
dummies_source = pd.get_dummies(df_con['source'], prefix= 'source')
#dummies_device = pd.get_dummies(df_con['device'], prefix= 'device')
#dummies_system = pd.get_dummies(df_con['operative_system'], prefix= 'system')
#df_control = pd.concat([df_con, dummies_source, dummies_device, dummies_system], axis=1)
df_control = pd.concat([df_con, dummies_source], axis=1)
df_control.drop(['source', 'device', 'operative_system'], axis=1, inplace=True)
# df_control 

In [95]:
# one hot encoding the experiment table
dummies_source = pd.get_dummies(df_exp['source'], prefix= 'source')
dummies_device = pd.get_dummies(df_exp['device'], prefix= 'device')
dummies_system = pd.get_dummies(df_exp['operative_system'], prefix= 'system')
#df_experiment = pd.concat([df_exp, dummies_source, dummies_device, dummies_system], axis=1)
df_experiment = pd.concat([df_exp, dummies_source], axis=1)

df_experiment.drop(['source', 'device', 'operative_system'], axis=1, inplace=True)
df_experiment  

,timestamp,test,price,converted,city,country,lat,long,net_revenue,source_ads-bing,source_ads-google,source_ads-yahoo,source_ads_facebook,source_ads_other,source_direct_traffic,source_friend_referral,source_seo-bing,source_seo-google,source_seo-other,source_seo-yahoo,source_seo_facebook
user_id,,,,,,,,,,,,,,,,,,,,,
685636,2015-05-07 07:26:01,1,59,0,Fayetteville,USA,35.07,-78.90,0,0,0,0,0,0,1,0,0,0,0,0,0
798371,2015-03-15 08:19:29,1,59,1,East Orange,USA,40.77,-74.21,59,1,0,0,0,0,0,0,0,0,0,0,0
447194,2015-03-28 12:28:10,1,59,0,Dayton,USA,39.78,-84.20,0,0,0,0,1,0,0,0,0,0,0,0,0
431639,2015-04-24 12:42:18,1,59,0,Richmond,USA,37.53,-77.47,0,0,0,0,1,0,0,0,0,0,0,0,0
552048,2015-03-22 08:58:32,1,59,0,Culver City,USA,34.01,-118.40,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190563,2015-05-17 12:03:19,1,59,0,New Haven,USA,41.31,-72.92,0,0,0,0,0,0,0,0,0,0,0,0,1
796427,2015-04-02 09:33:18,1,59,0,Norwalk,USA,33.91,-118.08,0,0,0,0,0,0,0,0,0,1,0,0,0
964001,2015-05-05 13:31:19,1,59,0,Athens-Clarke,USA,33.95,-83.39,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
# count metrics
results = {"Control":pd.Series(df_control.iloc[:, -19:-1].sum(axis=0)),
           "Experiment":pd.Series(df_experiment.iloc[:, -19:-1].sum(axis=0))}
df_results = pd.DataFrame(results)
df_results

In [ ]:
# Count Metrics
df_results['Total']=df_results.Control + df_results.Experiment
df_results['Prob'] = 0.36
df_results['StdErr'] = np.sqrt((df_results.Prob * (1- df_results.Prob))/df_results.Total)
df_results["MargErr"] = 1.96 * df_results.StdErr
df_results["CI_lower"] = df_results.Prob - df_results.MargErr
df_results["CI_upper"] = df_results.Prob + df_results.MargErr
df_results["Obs_val"] = df_results.Experiment/df_results.Total
df_results["Pass_Sanity"] = df_results.apply(lambda x: (x.Obs_val > x.CI_lower) and (x.Obs_val < x.CI_upper),axis=1)
df_results['Diff'] = abs((df_results.Experiment - df_results.Control)/df_results.Total)

df_results

As seen from the invariant metrics sanity check result table, it can see that most of the source have pass the sanity check, but the device and system are not invariant.

### Result Analysis
95% Confidence interval for the difference between the experiment and control group for evaluation metrics. The result is satistically significant only when the 95% confidence interval does not include zero.

In [ ]:
# get the converted results table
df_control_notnull = df_control[pd.isnull(df_control.converted) != True]
df_experiment_notnull = df_experiment[pd.isnull(df_experiment.converted) != True]

In [ ]:
results_notnull = {"Control":pd.Series(df_control.iloc[:, -19:-1].sum(axis=0)),
           "Experiment":pd.Series(df_experiment.iloc[:, -19:-1].sum(axis=0))}
df_results_notnull = pd.DataFrame(results_notnull)
df_results_notnull['Total']=df_results_notnull.Control + df_results_notnull.Experiment
df_results_notnull

In [ ]:
def stats_prop(p_hat,z_score,N_cont,N_exp,diff):
    std_err = np.sqrt((p_hat * (1- p_hat ))*(1/N_cont + 1/N_exp))
    marg_err = z_score * std_err
    ci_lower = diff - marg_err
    ci_upper = diff + marg_err
    
    return std_err,marg_err,ci_lower,ci_upper

In [ ]:
## Comparing two samples
# experimental values
converted_exp = df_results_notnull.loc["converted"].Experiment/N_exp
rev_exp = df_results_notnull.loc["net_revenue"].Experiment/N_exp

# control values
converted_cont = df_results_notnull.loc["converted"].Control/N_con
rev_cont = df_results_notnull.loc["net_revenue"].Control/N_con

# get the differences
converted_diff = converted_exp - converted_cont
rev_diff = rev_exp - rev_cont
print(converted_diff, rev_diff)

# The pooled probability
Conversion = (converted_exp + converted_cont)/(N_exp + N_con)
NetRevenue = (rev_exp + rev_cont)/(N_exp + N_con)

# Compare two samples
se_Conversion,me_Conversion,cil_Conversion,ciu_Conversion = stats_prop(Conversion, 1.96, N_con, N_exp, converted_diff)
print(se_Conversion,me_Conversion,cil_Conversion,ciu_Conversion)

se_NetRevenue,me_NetRevenue,cil_NetRevenue,ciu_NetRevenue = stats_prop(NetRevenue, 1.96, N_con, N_exp, rev_diff)
print(se_NetRevenue,me_NetRevenue,cil_NetRevenue,ciu_NetRevenue)

<table style="width:100%;">
<colgroup>
<col width="12%" />
<col width="23%" />
<col width="23%" />
<col width="23%" />
<col width="23%" />
<col width="12%" />
</colgroup>
<thead>
<tr class="header">
<th align="center">Metric</th>
<th align="center">dmin</th>
<th align="center">Observed Difference</th>
<th align="center">CI Lower Bound</th>
<th align="center">CI Upper Bound</th>
<th align="center">Result</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td align="center">Conversion</td>
<td align="center">0.01</td>
<td align="center">-0.0043</td>
<td align="center">-0.004347</td>
<td align="center">-0.004342</td>
<td align="center">Satistically and Practically Significant</td>
</tr>
<tr class="even">
<td align="center">Net Revenue</td>
<td align="center">0.0075</td>
<td align="center">0.141665</td>
<td align="center">0.141648</td>
<td align="center">0.141682</td>
<td align="center">Statistically and Practically Significant</td>
</tr>
</tbody>
</table>